In [16]:
import zipfile
import pandas as pd

# Unzip
with zipfile.ZipFile("archive (36).zip", 'r') as zip_ref:
    zip_ref.extractall("tiny_stories")



In [17]:
import pandas as pd

# Load the file
df = pd.read_csv("tiny_stories/Untitled form.csv")

# Preview first few rows
df.head()


,Timestamp,1.Are you familiar with the concept of autonomous vehicles?,2.Do you think autonomous vehicles should prioritize the safety of their occupants over pedestrians and other road users?,"3.In a hypothetical scenario, an autonomous vehicle has to choose between hitting a pedestrian crossing the road illegally and swerving into a wall, potentially injuring the vehicle's occupants. What do you think the autonomous vehicle should do?",4.Do you believe that the introduction of autonomous vehicles will lead to a decrease in overall traffic accidents and fatalities?,5.Who do you believe should be held liable in the event of an accident involving an autonomous vehicle?,6.How concerned are you about the potential privacy and cyber security risks associated with autonomous vehicles?,7.Do you believe the widespread adoption of autonomous vehicles could lead to job displacement in the transportation industry?
0,2023/08/05 4:22:58 PM GMT+1,Yes,Yes,Prioritize protecting the occupants,Yes,Not sure,Very concerned,Yes
1,2023/08/05 4:25:11 PM GMT+1,Yes,Yes,Not sure,Yes,Not sure,Somewhat concerned,No
2,2023/08/05 4:26:28 PM GMT+1,Yes,No,Prioritize protecting the pedestrians,No,The vehicle owner,Neutral,Not sure
3,2023/08/05 4:32:22 PM GMT+1,Yes,Not sure,Prioritize protecting the pedestrians,No,Not sure,Neutral,Yes
4,2023/08/05 4:44:06 PM GMT+1,Yes,Yes,Prioritize protecting the pedestrians,Yes,The autonomous vehicle manufacturer,Somewhat concerned,It's hard to predict


In [18]:
print(df.columns)


Index(['Timestamp',
       '1.Are you familiar with the concept of autonomous vehicles?',
       '2.Do you think autonomous vehicles should prioritize the safety of their occupants over pedestrians and other road users?',
       '3.In a hypothetical scenario, an autonomous vehicle has to choose between hitting a pedestrian crossing the road illegally and swerving into a wall, potentially injuring the vehicle's occupants. What do you think the autonomous vehicle should do?',
       '4.Do you believe that the introduction of autonomous vehicles will lead to a decrease in overall traffic accidents and fatalities?',
       '5.Who do you believe should be held liable in the event of an accident involving an autonomous vehicle?',
       '6.How concerned are you about the potential privacy and cyber security risks associated with autonomous vehicles?',
       '7.Do you believe the widespread adoption of autonomous vehicles could lead to job displacement in the transportation industry?'],
    

In [19]:
df = pd.read_csv("tiny_stories/Untitled form.csv")

# Rename for clarity
df = df.rename(columns={
    "3.In a hypothetical scenario, an autonomous vehicle has to choose between hitting a pedestrian crossing the road illegally and swerving into a wall, potentially injuring the vehicle's occupants. What do you think the autonomous vehicle should do?": "text"
})

# Drop rows with empty responses
df = df.dropna(subset=["text"])

# Optional: Preview unique responses
print(df["text"].value_counts())


text
Prioritize protecting the pedestrians    18
Prioritize protecting the occupants      15
Not sure                                 12
Randomly decide                           4
Name: count, dtype: int64


In [20]:
# Map text responses to labels
label_map = {
    "Prioritize protecting the pedestrians": 0,
    "Prioritize protecting the occupants": 1,
    "Not sure": 2,
    "Randomly decide": 2  # Merge both into "uncertain" category
}

df = df[df["text"].isin(label_map.keys())]
df["label"] = df["text"].map(label_map)


In [21]:
scenario = (
    "In a hypothetical scenario, an autonomous vehicle has to choose between "
    "hitting a pedestrian crossing the road illegally and swerving into a wall, "
    "potentially injuring the vehicle's occupants. What do you think the autonomous vehicle should do?"
)

df["text"] = scenario  # Overwrite with common scenario


In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from transformers import TrainingArguments, Trainer

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df[["text", "label"]])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the dataset
tokenized_dataset = hf_dataset.map(
    lambda x: tokenizer(x["text"], padding="max_length", truncation=True),
    batched=True
)


C:\Users\Sreeja Mondal\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/49 [00:43<?, ? examples/s]

In [23]:
pip install accelerate -U


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [24]:
pip install -U accelerate


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
!pip install --upgrade accelerate


Defaulting to user installation because normal site-packages is not writeable


In [27]:
import accelerate
print(accelerate.__version__)  # should be >= 0.21.0


1.9.0


In [28]:
!pip install --upgrade transformers


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
    --------------------------------------- 0.3/11.2 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.2 MB 1.9 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.2 MB 2.8 MB/s eta 0:00:04
   --------- ------------------------------ 2.6/11.2 MB 3.4 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/11.2 MB 4.0 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.2 MB 3.6 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.2 MB 3.8 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.2 MB 4.0 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/11.2 MB 4.2 MB/s eta 0:00:01
   ------------------------------- -------- 8.9/11.2 MB 4.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.2 MB 4.5 MB/s eta 0:00:01
   --------

  You can safely remove it manually.


In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AdamW

# 1. Wrap tokenized dataset in PyTorch Dataset
class MoralDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.input_ids = tokenized_dataset["input_ids"]
        self.attention_mask = tokenized_dataset["attention_mask"]
        self.labels = tokenized_dataset["label"]

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_mask[idx]),
            "labels": torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

# 2. Create PyTorch dataloader
train_dataset = MoralDataset(tokenized_dataset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# 3. Load model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# 4. Choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 5. Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# 6. Training loop
model.train()
epochs = 5

for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} — Loss: {avg_loss:.4f}")


C:\Users\Sreeja Mondal\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Sreeja Mondal\AppData\Roaming\Python\Python312\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  lr_lambda 

Epoch 1 — Loss: 1.1216
Epoch 2 — Loss: 1.0914
Epoch 3 — Loss: 1.1202
Epoch 4 — Loss: 1.1050
Epoch 5 — Loss: 1.0906


In [33]:
print(len(df))  


49


In [34]:
print(df['label'].value_counts())


label
0    18
2    16
1    15
Name: count, dtype: int64


In [35]:
augmented_data = {
    "Prioritize protecting the pedestrians": [
        "The vehicle should ensure pedestrian safety at all costs.",
        "Pedestrians must be protected even if passengers are at risk.",
        "Always prioritize pedestrians, regardless of circumstance.",
        "In any case, save the pedestrian's life first."
    ],
    "Prioritize protecting the occupants": [
        "The passengers inside the vehicle should be kept safe.",
        "Saving the lives of the vehicle's occupants is crucial.",
        "The safety of those inside the car comes first.",
        "Prioritize the people in the car over others."
    ],
    "Not sure": [
        "It’s difficult to make a clear decision here.",
        "There’s no definitive right answer in this scenario.",
        "I’m uncertain about the best course of action.",
        "This situation is too complex to decide."
    ],
    "Randomly decide": [
        "Let the vehicle make a random choice.",
        "Either outcome is acceptable; pick randomly.",
        "The car should decide arbitrarily.",
        "Random decision-making might be the fairest."
    ]
}


In [36]:
import pandas as pd

# Label mapping
label_map = {
    "Prioritize protecting the pedestrians": 0,
    "Prioritize protecting the occupants": 1,
    "Not sure": 2,
    "Randomly decide": 2
}

# Build new dataset
new_rows = []
for original_text, variations in augmented_data.items():
    for variant in variations:
        new_rows.append({
            "text": variant,
            "label": label_map[original_text]
        })

aug_df = pd.DataFrame(new_rows)

# Combine with your original df
final_df = pd.concat([df[["text", "label"]], aug_df], ignore_index=True)

print(f"New dataset size: {len(final_df)}")


New dataset size: 65


In [37]:
pedestrian_variants = [
    "The lives of pedestrians should be the top priority.",
    "The car should always avoid harming people on foot.",
    "It must swerve or stop to protect pedestrians.",
    "Pedestrians deserve maximum protection in all situations.",
    "The vehicle should favor saving people outside it.",
    "No matter what, pedestrians must be spared.",
    "Road users outside the car should be prioritized."
]


In [38]:
occupant_variants = [
    "Preserving the passengers’ lives is more important.",
    "The car should never endanger those inside it.",
    "Occupants should be protected first and foremost.",
    "The safety of riders is the top concern.",
    "Saving the car's passengers takes precedence.",
    "Focus on those inside the vehicle.",
    "People in the vehicle should come first."
]


In [39]:
uncertain_variants = [
    "It’s hard to say what the right decision is.",
    "Both outcomes seem equally difficult to justify.",
    "I can’t determine the best ethical choice here.",
    "Maybe the car should just choose randomly.",
    "There’s no morally perfect decision here.",
    "Tough call – not sure what the right thing is.",
    "Let the system decide based on chance."
]


In [40]:
extra_data = []

for sentence in pedestrian_variants:
    extra_data.append({"text": sentence, "label": 0})
for sentence in occupant_variants:
    extra_data.append({"text": sentence, "label": 1})
for sentence in uncertain_variants:
    extra_data.append({"text": sentence, "label": 2})

extra_df = pd.DataFrame(extra_data)
final_df = pd.concat([final_df, extra_df], ignore_index=True)
print(f"Expanded dataset size: {len(final_df)}")


Expanded dataset size: 86


In [41]:
from datasets import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
hf_dataset = Dataset.from_pandas(final_df)

tokenized_dataset = hf_dataset.map(
    lambda x: tokenizer(x["text"], padding="max_length", truncation=True),
    batched=True
)


C:\Users\Sreeja Mondal\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/86 [00:00<?, ? examples/s]

In [42]:
epochs = 10  


In [43]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSequenceClassification

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert Hugging Face dataset to PyTorch dataset
class MoralDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

# Prepare inputs
tokenized_dataset.set_format("torch")
dataset = MoralDataset(tokenized_dataset, tokenized_dataset["label"])
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3
).to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training loop
epochs = 10
model.train()

for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} — Loss: {avg_loss:.4f}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Sreeja Mondal\AppData\Local\Temp\ipykernel_9620\4236131529.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
C:\Users\Sreeja Mondal\AppData\Local\Temp\ipykernel_9620\4236131529.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(self.enco

Epoch 1 — Loss: 1.1008
Epoch 2 — Loss: 0.9982
Epoch 3 — Loss: 0.8817
Epoch 4 — Loss: 0.7582
Epoch 5 — Loss: 0.7012
Epoch 6 — Loss: 0.6543
Epoch 7 — Loss: 0.6458
Epoch 8 — Loss: 0.6492
Epoch 9 — Loss: 0.6293
Epoch 10 — Loss: 0.6466


In [44]:
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in train_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(inputs["labels"].cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=["Pedestrian", "Occupant", "Uncertain"]))


C:\Users\Sreeja Mondal\AppData\Local\Temp\ipykernel_9620\4236131529.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
C:\Users\Sreeja Mondal\AppData\Local\Temp\ipykernel_9620\4236131529.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
C:\Users\Sreeja Mondal\AppData\Local\Temp\ipykernel_9620\4236131529.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(self.labels[idx])


              precision    recall  f1-score   support

  Pedestrian       0.48      1.00      0.65        29
    Occupant       1.00      0.42      0.59        26
   Uncertain       1.00      0.48      0.65        31

    accuracy                           0.64        86
   macro avg       0.83      0.64      0.63        86
weighted avg       0.83      0.64      0.63        86

